# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [ ]:
class Matrix:

    def __init__(self, n, m, value=0):
        self.__n = n
        self.__m = m
        self.M= [[value] * m for _ in range(n)]
    
    def __init__(self, data):
        self.__n= len(data)
        self.__m = len(data[0])
        self.M = data

    def __getitem__(self, index):
        if isinstance(index, tuple):
            i, j = index
            return self.M[i][j]
        elif isinstance(index, slice):
            return Matrix(self.M[index])
        elif isinstance(index, int):
            return self.M[index]
        else:
            raise TypeError("Invalid index type")
    
    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            i, j = index
            self.M[i][j] = value
        elif isinstance(index, int):
            self.M[index] = value
        else:
            raise TypeError("Invalid index type")
        
    def __str__(self):
        result = ""
        for row in self.M:
            result += ' '.join(map(str, row)) + '\n'
        return result.strip()
    
    def shape(self):
        return (self.__n, self.__m)
    
    def transpose(self):
        transposed = [[self.M[j][i] for j in range(self.__n)] for i in range(self.__m)]
        return Matrix(transposed)

    def row(self, i):
        return Matrix(self.M[i])
    
    def column(self, j):
        col = [[self.M[i][j]] for i in range(self.__n)]
        return Matrix(col)
    
    def to_list(self):
        return self.M

    def block(self, n_0, n_1, m_0, m_1):
        block = [row[m_0:m_1] for row in self.M[n_0:n_1]]
        return Matrix(block)
    
    def constant(self, n, m, c):
        return Matrix(n, m, c)
    
    def zeroes(self, n, m):
        return Matrix(n, m)
    
    def ones(self, n, m):
        return Matrix(n, m, 1)
    
    def eye(self, n):
        eye = Matrix(n, n)
        for i in range(n):
            eye[i, i] = 1
        return eye

    def scalarmul(self, c):
        return [[c * self.M[i][j] for j in range(self.__m)] for i in range(self.__n)])




mat = Matrix([[1, 2], [4, 5], [7, 8]])
n, m = mat.shape()
# Test block





Block (0, 2, 0, 2):
1 2
4 5
